<a href="https://colab.research.google.com/github/AgomferAustral/DMA-Caras/blob/main/aumentacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# prompt: abrir google drive

from google.colab import drive

# Unmount the drive first if it is mounted
try:
  drive.flush_and_unmount()
except ValueError:
  pass

# Mount the drive to the desired path
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# List files in your Drive
!ls /content/drive/MyDrive


Mounted at /content/drive
 BackPropagation.ipynb	 Detectadas   'Hoja de cálculo sin título.gsheet'
 caras			 Detectadas2   labo1
'Colab Notebooks'	 DMA	       shape_predictor_68_face_landmarks.dat
 datos_isomap.pkl	 fotos	       viejos


In [21]:
import os
from PIL import Image, ImageOps
from pathlib import Path



In [22]:
# Carpeta de origen y destino
carpeta_origen = Path("/content/drive/MyDrive/Detectadas")
carpeta_destino = Path("/content/drive/MyDrive/Aumentadas")


In [23]:
# Crear carpeta destino si no existe
carpeta_destino.mkdir(parents=True, exist_ok=True)

# Transformaciones
desplazamientos = [-10, 10]  # en px
rotaciones = [-10, 10]  # en grados

def desplazar(img, dx=0, dy=0):
    """Desplaza una imagen dentro del lienzo original."""
    fondo = Image.new("L", img.size, color=0)  # Asumimos imágenes en escala de grises
    fondo.paste(img, (dx, dy))
    return fondo

def guardar_variacion(imagen, nombre_base, sufijo, carpeta):
    """Guarda la imagen con sufijo en la carpeta destino."""
    nombre_archivo = f"{nombre_base}_{sufijo}.png"
    ruta_guardado = carpeta / nombre_archivo
    imagen.save(ruta_guardado)


In [24]:
# version original con listas reducidas

"""
# Recorrer subcarpetas e imágenes
for subcarpeta in carpeta_origen.iterdir():
    if subcarpeta.is_dir():
        sub_destino = carpeta_destino / subcarpeta.name
        sub_destino.mkdir(parents=True, exist_ok=True)

        for archivo in subcarpeta.glob("*.jp*g"):  # Suponiendo imágenes .jpg
            imagen = Image.open(archivo)
            nombre_base = archivo.stem

            # Guardar imagen original
            imagen.save(sub_destino / f"{nombre_base}_original.jpg")

            variaciones = []

            # a) Desplazamientos horizontales
            for dx in [-10, 10]:
                img_dx = desplazar(imagen, dx=dx)
                guardar_variacion(img_dx, nombre_base, f"dx{dx}", sub_destino)
                variaciones.append((img_dx, f"dx{dx}"))

            # b) Desplazamientos verticales
            for dy in [-10,  10]:
                img_dy = desplazar(imagen, dy=dy)
                guardar_variacion(img_dy, nombre_base, f"dy{dy}", sub_destino)
                variaciones.append((img_dy, f"dy{dy}"))

            # c) Reflejo horizontal
            img_flip = ImageOps.mirror(imagen)
            guardar_variacion(img_flip, nombre_base, "flip", sub_destino)
            variaciones.append((img_flip, "flip"))

            # d) Rotaciones sobre todas las nuevas imágenes
            for img, sufijo in variaciones + [(imagen, "original")]:
                for ang in [-10, 10]:
                    img_rot = img.rotate(ang, expand=True, fillcolor=0)
                    guardar_variacion(img_rot, nombre_base, f"{sufijo}_rot{ang}", sub_destino)
"""

'\n# Recorrer subcarpetas e imágenes\nfor subcarpeta in carpeta_origen.iterdir():\n    if subcarpeta.is_dir():\n        sub_destino = carpeta_destino / subcarpeta.name\n        sub_destino.mkdir(parents=True, exist_ok=True)\n\n        for archivo in subcarpeta.glob("*.jp*g"):  # Suponiendo imágenes .jpg\n            imagen = Image.open(archivo)\n            nombre_base = archivo.stem\n\n            # Guardar imagen original\n            imagen.save(sub_destino / f"{nombre_base}_original.jpg")\n\n            variaciones = []\n\n            # a) Desplazamientos horizontales\n            for dx in [-10, 10]:\n                img_dx = desplazar(imagen, dx=dx)\n                guardar_variacion(img_dx, nombre_base, f"dx{dx}", sub_destino)\n                variaciones.append((img_dx, f"dx{dx}"))\n\n            # b) Desplazamientos verticales\n            for dy in [-10,  10]:\n                img_dy = desplazar(imagen, dy=dy)\n                guardar_variacion(img_dy, nombre_base, f"dy{dy}",

In [25]:
#nueva version

# Recorrer subcarpetas e imágenes
for subcarpeta in carpeta_origen.iterdir():
    if subcarpeta.is_dir():
        sub_destino = carpeta_destino / subcarpeta.name
        sub_destino.mkdir(parents=True, exist_ok=True)

        for archivo in subcarpeta.glob("*.jp*g"):  # Suponiendo imágenes .jpg
            imagen = Image.open(archivo).convert("L") # Ensure grayscale for consistent displacement background
            nombre_base = archivo.stem

            # Get image dimensions for percentage calculations
            ancho, alto = imagen.size
            desplazamiento_horizontal = int(ancho * 0.05)
            desplazamiento_vertical = int(alto * 0.05)

            # Guardar imagen original
            #imagen.save(sub_destino / f"{nombre_base}_original.jpg")

            # --- Apply transformations based on the original image ---

            # a) Desplazamientos horizontales (5% izquierda y derecha)
            img_dx_left = desplazar(imagen, dx=-desplazamiento_horizontal)
            guardar_variacion(img_dx_left, nombre_base, f"dx{-int(desplazamiento_horizontal/ancho*100)}pct", sub_destino)

            img_dx_right = desplazar(imagen, dx=desplazamiento_horizontal)
            guardar_variacion(img_dx_right, nombre_base, f"dx{int(desplazamiento_horizontal/ancho*100)}pct", sub_destino)

            # b) Desplazamientos verticales (5% arriba y abajo)
            img_dy_up = desplazar(imagen, dy=-desplazamiento_vertical)
            guardar_variacion(img_dy_up, nombre_base, f"dy{-int(desplazamiento_vertical/alto*100)}pct", sub_destino)

            img_dy_down = desplazar(imagen, dy=desplazamiento_vertical)
            guardar_variacion(img_dy_down, nombre_base, f"dy{int(desplazamiento_vertical/alto*100)}pct", sub_destino)

            # c) Rotaciones (10 grados horario y antihorario)
            img_rot_neg10 = imagen.rotate(-10, expand=True, fillcolor=0)
            guardar_variacion(img_rot_neg10, nombre_base, "rot-10", sub_destino)

            img_rot_pos10 = imagen.rotate(10, expand=True, fillcolor=0)
            guardar_variacion(img_rot_pos10, nombre_base, "rot10", sub_destino)

            # Note: We removed the horizontal flip and the subsequent rotations
            # on variations as per your request to only use the original image.

In [26]:
Stop

NameError: name 'Stop' is not defined

# No estoy utilizando estos otros efectos

# Efecto ojo de pez

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('/content/drive/MyDrive/origen/sujeto2/foto.jpg')
destino=Path("/content/drive/MyDrive/destino/sujeto2")

# Crear carpeta destino si no existe
destino.mkdir(parents=True, exist_ok=True)

img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

plt.imshow(img)
plt.title("Imagen original")
plt.axis('off')
plt.show()

h, w = img.shape[:2]
K = np.array([[w, 0, w/2],
              [0, w, h/2],
              [0, 0, 1]], dtype=np.float32)

valores = [-0.3,-0.1, 0, 0.1, 0.3]
valores2 = [-0.1, 0, 0.1]

for i in valores:
  for j in valores:
    for k in valores2:
      for l in [0]:
        for m in [0]:
          # Coeficientes de distorsión (k1, k2, p1, p2, k3)
          dist_coeffs = np.array([i, j, k, l, m])  # Ajustá para intensificar o reducir

          # Aplicar distorsión
          map1, map2 = cv2.initUndistortRectifyMap(K, dist_coeffs, None, K, (w, h), cv2.CV_32FC1)
          fisheye_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR)

          nombre_archivo = f"foto_{i}{j}{k}.jpg"
          print(nombre_archivo)
          ruta_guardado = f"{destino}/{nombre_archivo}"
          cv2.imwrite(ruta_guardado, fisheye_img)



# Efecto zoom / teleobjetivo

In [ ]:
import cv2
import matplotlib.pyplot as plt

def zoom_center(img, zoom_factor=2.0):
    h, w = img.shape[:2]
    new_h, new_w = int(h / zoom_factor), int(w / zoom_factor)

    # Coordenadas del recorte central
    start_y = (h - new_h) // 2
    start_x = (w - new_w) // 2

    # Recortar y redimensionar al tamaño original
    cropped = img[start_y:start_y+new_h, start_x:start_x+new_w]
    zoomed = cv2.resize(cropped, (w, h), interpolation=cv2.INTER_LINEAR)

    return zoomed

# Cargar imagen
img = cv2.imread("/content/drive/MyDrive/origen/sujeto2/foto.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Aplicar zoom
zoom_img = zoom_center(img, zoom_factor=1.2)

# Mostrar
plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.title("Original")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(zoom_img)
plt.title("Zoom / Teleobjetivo")
plt.axis('off')
plt.show()


# Efecto zoom a toda la carpeta

In [ ]:
import os
import cv2
from pathlib import Path

def zoom_center(img, zoom_factor=1.2):
    """Applies zoom effect to the center of an image."""
    h, w = img.shape[:2]
    new_h, new_w = int(h / zoom_factor), int(w / zoom_factor)
    start_y = (h - new_h) // 2
    start_x = (w - new_w) // 2
    cropped = img[start_y:start_y+new_h, start_x:start_x+new_w]
    zoomed = cv2.resize(cropped, (w, h), interpolation=cv2.INTER_LINEAR)
    return zoomed

# Define source and destination folders
carpeta_origen = Path("/content/drive/MyDrive/destino")  # Source folder
carpeta_destino = Path("/content/drive/MyDrive/destino2") # Destination folder

# Create destination folder if it doesn't exist
carpeta_destino.mkdir(parents=True, exist_ok=True)

# Iterate through subfolders and images
for subcarpeta in carpeta_origen.iterdir():
    if subcarpeta.is_dir():
        # Create corresponding subfolder in destination
        sub_destino = carpeta_destino / subcarpeta.name
        sub_destino.mkdir(parents=True, exist_ok=True)

        for archivo in subcarpeta.glob("*.jp*g"):  # Assuming .jpg or .jpeg images
            # Load image in grayscale
            img = cv2.imread(str(archivo), cv2.IMREAD_GRAYSCALE)

            # Apply zoom effect
            zoom_factor = 1.2
            zoom_img = zoom_center(img,1.5)

            # Save zoomed image in destination folder
            nombre_archivo = f"{zoom_factor}{archivo.name}"  # Keep original filename
            ruta_guardado = sub_destino / nombre_archivo
            cv2.imwrite(str(ruta_guardado), zoom_img)

print("Zoom effect applied and images saved to 'destino2' folder.")